---

# Applied Economic Analysis 1: Python assignment

|Name|SNR|ANR|
|----|---|----|
|Miquel Soler|2063831|u700978|

# Research question 

In this notebook I will compose an index measuring the best European trade partners for each country. A large amount of trade can be simply explained through the Gravity model of trade, which, just like the Gravity in physics, it is only concerned in mass (size of the economy) and the distance from each other. Therefore, as both variables have an exogenous nature, I will be creating an index showing the amount of trade for each European country after controlling for the effect of the Gravity model. By doing this, I hope to be able to show country levels of trade despite their natural trade barriers, and therefore be able to create an index showing the countries which have the best and worst trade relationships with each other.

# Motivation 

My main motivation on creating this index came from the following map: <img src="MajorTradingPartners.jpg">
Author: u/JohnPaokJeff

The map shows the major trading partner in Europe and its surrounding areas, however it does not provide much insight as it is simply the result of the Gravity model: countries trade most with the country with the largest economy which is relatively closer to them.
Therefore, I believe controlling for this effect would lead to a much more insightful map as it would show the country's biggest trade partners based on how good their trade relationships are instead of simply their size and proximity. 

Being able to display each country's best and worst trade partners would be insightful as it would illustrate the complex environment of international trade within a simple ranking system. Furthermore, I expect many smaller countries to unexpectedly show up high on a country's rankings as these are normally overshadowed by the main economic powerhouses of Europe.

# Method 

After consolidating our data, we will be conducting an Ordinary Least Squares regression in order to capture the effect of the Gravity Equation on international trade within Europe.

Once we have obtained the coefficients of our model, we will use them to calculate the deviation from our model from each country's trading relationships in order to rank each country based on the difference between the real trade figures and those estimated from our model.

After carrying out this procedure, we will finally recreate the map above while having controlled for the effect of the Gravity Equation. The final map will be done through Adobe Photoshop.

# Data 

The scope of this project is limited to European countries. 
However, due to data unavailability the countries of Andorra, Liechtenstein, Monaco, San Marino, and the Vatican City have been excluded.

We will be using data from three different sources:

* We download trade balance, exports and imports for each country from the World Integrated Trade Solution, a trade software provided by the World Bank
<br>link for the Netherlands: https://wits.worldbank.org/CountryProfile/en/Country/NLD/Year/2018/TradeFlow/EXPIMP (we download a file for each of our 40 countries)

* Using the wbdata library, we retrieve GDP (Current USD 2018) data through their API
<br>wbdata documentation: https://wbdata.readthedocs.io/en/stable/

* In order to calculate distances between countries we will need to retrieve the coordinates of the center each country.
<br>However, we are not interested in the geographical center and would the coordinates of the center of their economy. As this is unavailable, we will approximate this by using the population-weighted centers of each European country using a study from Baylor University. 
As this is unavailable for Serbia and Montenegro as they are displayed as one country, we will use the coordinates of the center of their respective capital cities.
Another concern is that the data is based on data from 2005. However, the coordinates do not seem to move much from 1990 to 2005. Therefore, we expect that the outdated data will have an unsignificant effect on our model.
paper: http://cs.ecs.baylor.edu/~hamerly/software/europe_population_weighted_centers.html
results: http://cs.ecs.baylor.edu/~hamerly/software/europe_population_weighted_centers.txt

We will then consolidate our data through pandas by selecting the rows (we will exclude all other countries which are not in our list from our rows) and columns (we will select the variables for GDP, Imports, Exports, and coordinates). For our coordinates, we will end up using them within a function in order to retrieve the distance value in km for each pair of countries. After retrieving their distance, we will introduce this into our main dataframe and drop the coordinates behind.

After consolidating our data, we will be conducting an Ordinary Least Squares regression in order to capture the effect of the Gravity Equation on international trade within Europe.

Once we have obtained the coefficients of our model, we will use them to calculate the deviation from our model from each country's trading relationships in order to rank each country based on the difference between the real trade figures and those estimated from our model.

# Preview of the answers

We summarize the results through the following map. 
<br>Each country's best trading partner is represented by their flag 
<br>(i.e., the best trading partner of the Netherlands is Iceland)

## Best Trade Partner per Country (after controlling for economic size and distance):  
<img src="trademap.png">

# Main assumptions

Assumptions of our Model:
* GDP Current US\$ is a valid representation of Economic Size
* Exports + Imports are a valid representation of Trade
* The effects of distance affect each country equally
* Terrain: 
    - the effects a country being an island/landlocked is not considered)
    - the effects of mountain ranges and rivers are not considered

# Python code

### Importing Packages

Adjust the number of rows displayed by selecting the value on 'pd.set_option('display.max_rows', value)'

In [1]:
import pandas as pd
from pandas import DataFrame
import csv
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import wbdata
import datetime
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
pd.set_option('display.max_rows', 40)

### Defining Functions
<br>

**Distance**

In order to calculate our distances, we will need to use some trigonometry. The following code has been taken from: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude. A simple Pythagorean equation will not work as we need to consider the curvature of the Earth.

The code below has been confirmed to be accurate after cross-checking the results through the distance measurer in http://maps.google.com/

In [2]:
def distance(latA,lonA,latB,lonB):
    
    global dist
    # approximate radius of earth in km
    eRad = 6373.0

    lat1 = radians(latA)
    lon1 = radians(lonA)
    lat2 = radians(latB)
    lon2 = radians(lonB)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    dist = eRad * c
    #print(distance)

**GDP**

Using the wbdata library we can access the GDP values (current USD) for all of our countries.<br>
The code for this indicator is 'NY.GDP.MKTP.CD'
wbdata library documentation: https://wbdata.readthedocs.io/en/stable/

In [3]:
def country_data(country_code, indicator):

    data_dates = (datetime.datetime(2018,1,1), datetime.datetime(2018,1,1))
    #call the api
    data = wbdata.get_dataframe({indicator:'indicator'}, 
                                country=country_code, 
                                data_date=data_dates, 
                                convert_date=True, 
                                keep_levels=False)

    data = data.reset_index()
    #data = data.dropna() #if I want I can drop the na's
    return data['indicator'][0]

### Creating our Dataframes
<br>
We start by reading our 'coord.csv' csv file into a pandas dataframe (dfCoord) and sorting it by countries
<br>
By selecting the columns, we are interested in, we create a dataframe showing the coordinates for the population centroid of each country (dfDist)

In [4]:
dfCoord = pd.read_csv('coord.csv', delimiter=',')
dfCoord.sort_values('country', inplace=True)
dfDist = dfCoord[['country','latitude','longitude']]

In [5]:
dfDist #Run to view coordinates dataframe

,country,latitude,longitude
0,ALB,41.194385,19.911378
1,AUT,47.765194,14.634124
3,BEL,50.844560,4.437197
5,BGR,42.751707,25.087406
4,BIH,44.162657,17.761170
2,BLR,53.534381,27.811010
37,CHE,47.022583,7.953028
7,CYP,34.966787,33.257169
8,CZE,49.821788,15.613238
13,DEU,50.846581,9.686009


As this file only contains the countries we are interested in, we can use them to create our country list (countries) by selecting the 'country' column in the dataframe

In [6]:
countries = dfCoord['country']

In [7]:
countries #run to view the country list

0     ALB
1     AUT
3     BEL
5     BGR
4     BIH
2     BLR
37    CHE
7     CYP
8     CZE
13    DEU
9     DNK
35    ESP
10    EST
11    FIN
12    FRA
39    GBR
14    GRC
6     HRV
15    HUN
17    IRL
16    ISL
18    ITA
20    LTU
21    LUX
19    LVA
28    MDA
22    MKD
23    MLT
31    MNE
24    NLD
25    NOR
26    POL
27    PRT
29    ROM
30    RUS
32    SRB
33    SVK
34    SVN
36    SWE
38    UKR
Name: country, dtype: object

**We set our reporter to be the Netherlands and start creating a dataframe for the Netherlands which we can then use for our regression.**
- We fill the Country column with the Netherlands
- We fill the GDP_Reporter column with its value generated through the country_data function
- We fill in the Netherland's trade Partners through the countries list
- As we are not interested in the Netherland's trade with itself, we remove the row for which the Netherlands is its own trade partner
- Finally, we sort the dataframe alphabetically based on the trade partner list

In [8]:
data = {}
reporter = "NLD"
GDP = country_data(reporter,'NY.GDP.MKTP.CD')
df = pd.DataFrame(data)
df = df.assign(Country=[reporter]*40)
df = df.assign(GDP_Reporter=[GDP]*40)
df['Partner'] = countries
df = df[df.Partner != reporter]
df.sort_values('Partner', inplace=True)

In [9]:
df #run to view the dataframe at its current state

,Country,GDP_Reporter,Partner
0,NLD,9.140434e+11,ALB
1,NLD,9.140434e+11,AUT
3,NLD,9.140434e+11,BEL
5,NLD,9.140434e+11,BGR
4,NLD,9.140434e+11,BIH
2,NLD,9.140434e+11,BLR
37,NLD,9.140434e+11,CHE
7,NLD,9.140434e+11,CYP
8,NLD,9.140434e+11,CZE
13,NLD,9.140434e+11,DEU


- We now create a for loop in order to retrieve the GDP values for each country through the country_data function
- As we are looping through every country, including the Netherlands itself, it is important to exclude this row. We do this through an if statement making sure we only append data if the partner and reporter are not the same country
- And we save this as a list (partnerGDPlist) and then create a new column (GDP_Partner) on our dataframe with its values

In [10]:
partnerGDPlist = []
for country in countries:
    if country != reporter:
        partnerGDP = country_data(country,'NY.GDP.MKTP.CD')
        partnerGDPlist.append(partnerGDP) 
df['GDP_Partner'] = partnerGDPlist

In [11]:
df #run to view the dataframe at its current state

,Country,GDP_Reporter,Partner,GDP_Partner
0,NLD,9.140434e+11,ALB,1.514702e+10
1,NLD,9.140434e+11,AUT,4.550949e+11
3,NLD,9.140434e+11,BEL,5.437344e+11
5,NLD,9.140434e+11,BGR,6.623016e+10
4,NLD,9.140434e+11,BIH,2.018351e+10
2,NLD,9.140434e+11,BLR,6.003126e+10
37,NLD,9.140434e+11,CHE,7.051406e+11
7,NLD,9.140434e+11,CYP,2.530982e+10
8,NLD,9.140434e+11,CZE,2.489087e+11
13,NLD,9.140434e+11,DEU,3.963768e+12


- We now open the csv file 'WITS-Partner (NLD).csv' as downloaded from the wits.worldbank database for our current reporter (Netherlands)
- We replace the partner names by their Alpha-3 code so they match our dataframe
- We only include the export and import columns for the rows of the 40 countries we are interested in 
- We now add the exports and imports for each country and append them into a new list (tradeList)
- Finally, we add a new column to our dataframe (Trade) with the values from tradeList

In [12]:
file = 'WITS-Partner (%s).csv' %  reporter
dfTrade = pd.read_csv(file, delimiter=',')
dfTrade['Partner Name'] = dfTrade['Partner Name'].replace(['Albania','Austria','Belarus','Belgium','Bosnia and Herzegovina','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Iceland','Ireland','Italy','Latvia','Lithuania','Luxembourg','Macedonia','North Macedonia','Malta','Netherlands','Norway','Poland','Portugal','Moldova','Romania','Russia','Russian Federation','Montenegro','Serbia','Serbia, FR(Serbia/Montenegro)','Slovakia','Slovak Republic','Slovenia','Spain','Sweden','Switzerland','Ukraine','United Kingdom'],['ALB','AUT','BLR','BEL','BIH','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','ISL','IRL','ITA','LVA','LTU','LUX','MKD','MKD','MLT','NLD','NOR','POL','PRT','MDA','ROM','RUS','RUS','MNE','SRB','SRB','SVK','SVK','SVN','ESP','SWE','CHE','UKR','GBR'])
dfTrade = dfTrade[dfTrade["Partner Name"].isin(['ALB','AUT','BLR','BEL','BIH','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','ISL','IRL','ITA','LVA','LTU','LUX','MKD','MLT','NLD','NOR','POL','PRT','MDA','ROM','RUS','MNE','SRB','SVK','SVN','ESP','SWE','CHE','UKR','GBR'])]
dfTrade.sort_values('Partner Name', inplace=True)
tradeList = []
for country in countries:
    if country != reporter:
        dfTrade2 = dfTrade[['Partner Name','Export (US$ Thousand)','Import (US$ Thousand)']].loc[dfTrade['Partner Name'] == country]
        if (dfTrade2.iloc[0,1]):
            expimp = dfTrade2.iloc[0,1] + dfTrade2.iloc[0,2]
            tradeList.append(expimp)
df['Trade'] = tradeList

In [13]:
df #run to view the dataframe at its current state

,Country,GDP_Reporter,Partner,GDP_Partner,Trade
0,NLD,9.140434e+11,ALB,1.514702e+10,1.117953e+05
1,NLD,9.140434e+11,AUT,4.550949e+11,1.059512e+07
3,NLD,9.140434e+11,BEL,5.437344e+11,1.080948e+08
5,NLD,9.140434e+11,BGR,6.623016e+10,1.988375e+06
4,NLD,9.140434e+11,BIH,2.018351e+10,3.280845e+05
2,NLD,9.140434e+11,BLR,6.003126e+10,5.034814e+05
37,NLD,9.140434e+11,CHE,7.051406e+11,9.652560e+06
7,NLD,9.140434e+11,CYP,2.530982e+10,6.392941e+05
8,NLD,9.140434e+11,CZE,2.489087e+11,1.482135e+07
13,NLD,9.140434e+11,DEU,3.963768e+12,2.086668e+08


- We create a for loop to generate the distances between The Netherlands and our country list
- We create variables for the country's latitude and longitude values in our dfDist dataframe
- We calculate the distances through the distance function and append them into a new list (distanceList)
- Finally, we add a new column to our dataframe (Distance) with the values from distanceList

In [14]:
distanceList = []
for country in countries:
    if country != reporter:
        latReporter = dfDist.loc[dfDist['country'] == reporter].iloc[0,1]
        lonReporter = dfDist.loc[dfDist['country'] == reporter].iloc[0,2]
        latPartner = dfDist.loc[dfDist['country'] == country].iloc[0,1]
        lonPartner = dfDist.loc[dfDist['country'] == country].iloc[0,2]
        distance(latReporter,lonReporter,latPartner,lonPartner)
        distanceList.append(dist)
df['Distance'] = distanceList

We now have all the variables we need for the country of the Netherlands

In [15]:
df #run to view the dataframe at its final state

,Country,GDP_Reporter,Partner,GDP_Partner,Trade,Distance
0,NLD,9.140434e+11,ALB,1.514702e+10,1.117953e+05,1640.505182
1,NLD,9.140434e+11,AUT,4.550949e+11,1.059512e+07,822.094167
3,NLD,9.140434e+11,BEL,5.437344e+11,1.080948e+08,148.422748
5,NLD,9.140434e+11,BGR,6.623016e+10,1.988375e+06,1806.000820
4,NLD,9.140434e+11,BIH,2.018351e+10,3.280845e+05,1274.153539
2,NLD,9.140434e+11,BLR,6.003126e+10,5.034814e+05,1517.045534
37,NLD,9.140434e+11,CHE,7.051406e+11,9.652560e+06,593.378902
7,NLD,9.140434e+11,CYP,2.530982e+10,6.392941e+05,2918.937421
8,NLD,9.140434e+11,CZE,2.489087e+11,1.482135e+07,764.939015
13,NLD,9.140434e+11,DEU,3.963768e+12,2.086668e+08,333.870638


**Replicating these steps for all countries**
- Now that we have the data we need for the Netherlands; we will re-do all previous steps through a for loop for all countries in our list
- We also export our dataframe into a csv file in case we want to use this outside jupyterlab
- The following code will take some time to run as our dataframe will have 1560 rows × 6 columns 

In [16]:
df = {}
for reporter in countries:
    data = {}
    GDP = country_data(reporter,'NY.GDP.MKTP.CD')
    df[reporter] = pd.DataFrame(data)
    df[reporter] = df[reporter].assign(Country=[reporter]*40)
    df[reporter] = df[reporter].assign(GDP_Reporter=[GDP]*40)
    df[reporter]['Partner'] = countries
    df[reporter] = df[reporter][df[reporter].Partner != reporter]
    df[reporter].sort_values('Partner', inplace=True)
    
    partnerGDPlist = []
    for country in countries:
        if country != reporter:
            partnerGDP = country_data(country,'NY.GDP.MKTP.CD')
            partnerGDPlist.append(partnerGDP) 
    df[reporter]['GDP_Partner'] = partnerGDPlist
    file = 'WITS-Partner (%s).csv' %  reporter
    dfTrade = pd.read_csv(file, delimiter=',')
    dfTrade['Partner Name'] = dfTrade['Partner Name'].replace(['Albania','Austria','Belarus','Belgium','Bosnia and Herzegovina','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Iceland','Ireland','Italy','Latvia','Lithuania','Luxembourg','Macedonia','North Macedonia','Malta','Netherlands','Norway','Poland','Portugal','Moldova','Romania','Russia','Russian Federation','Montenegro','Serbia','Serbia, FR(Serbia/Montenegro)','Slovakia','Slovak Republic','Slovenia','Spain','Sweden','Switzerland','Ukraine','United Kingdom'],['ALB','AUT','BLR','BEL','BIH','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','ISL','IRL','ITA','LVA','LTU','LUX','MKD','MKD','MLT','NLD','NOR','POL','PRT','MDA','ROM','RUS','RUS','MNE','SRB','SRB','SVK','SVK','SVN','ESP','SWE','CHE','UKR','GBR'])
    dfTrade = dfTrade[dfTrade["Partner Name"].isin(['ALB','AUT','BLR','BEL','BIH','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','ISL','IRL','ITA','LVA','LTU','LUX','MKD','MLT','NLD','NOR','POL','PRT','MDA','ROM','RUS','MNE','SRB','SVK','SVN','ESP','SWE','CHE','UKR','GBR'])]
    dfTrade.sort_values('Partner Name', inplace=True)
    
    tradeList = []
    for country in countries:
        if country != reporter:
            dfTrade2 = dfTrade[['Partner Name','Export (US$ Thousand)','Import (US$ Thousand)']].loc[dfTrade['Partner Name'] == country]
            if (dfTrade2.iloc[0,1]):
                expimp = dfTrade2.iloc[0,1] + dfTrade2.iloc[0,2]
                tradeList.append(expimp)
    df[reporter]['Trade'] = tradeList
    
    distanceList = []
    for country in countries:
        if country != reporter:
            latReporter = dfDist.loc[dfDist['country'] == reporter].iloc[0,1]
            lonReporter = dfDist.loc[dfDist['country'] == reporter].iloc[0,2]
            latPartner = dfDist.loc[dfDist['country'] == country].iloc[0,1]
            lonPartner = dfDist.loc[dfDist['country'] == country].iloc[0,2]
            distance(latReporter,lonReporter,latPartner,lonPartner)
            distanceList.append(dist)
    df[reporter]['Distance'] = distanceList

dfAll = pd.concat(df)
dfAll.to_csv(r'df.csv')

In [17]:
dfAll #run to view the dataframe in its current state

Country  GDP_Reporter Partner   GDP_Partner        Trade     Distance
ALB 1      ALB  1.514702e+10     AUT  4.550949e+11     79968.33   841.882811
    3      ALB  1.514702e+10     BEL  5.437344e+11     37387.30  1600.602999
    5      ALB  1.514702e+10     BGR  6.623016e+10    109838.63   461.650831
    4      ALB  1.514702e+10     BIH  2.018351e+10     36665.88   374.025752
    2      ALB  1.514702e+10     BLR  6.003126e+10       194.90  1493.866592
...        ...           ...     ...           ...          ...          ...
UKR 30     UKR  1.309019e+11     RUS  1.669583e+12  11742819.78  1638.390411
    32     UKR  1.309019e+11     SRB  5.064065e+10    430954.93   967.319739
    33     UKR  1.309019e+11     SVK  1.057019e+11   1389805.76   921.085735
    34     UKR  1.309019e+11     SVN  5.416164e+10    223816.12  1296.938407
    36     UKR  1.309019e+11     SWE  5.554554e+11    535669.39  1537.059179

[1560 rows x 6 columns]

- Finally we can use the numpy.log() function to find the natural logarithm values of our variables and add these to our dataframe.

In [18]:
dfAll['const'] = 1
dfAll['l_Trade'] = np.log(dfAll['Trade'])
dfAll['l_Distance'] = np.log(dfAll['Distance'])
# dfAll['l_Distance_sq'] = np.log(dfAll['Distance']**2) Ended not using this variable on our model
# dfAll['Distance_sq'] = dfAll['Distance']**2 Ended not using this variable on our model
dfAll['l_GDP_Reporter'] = np.log(dfAll['GDP_Reporter'])
dfAll['l_GDP_Partner'] = np.log(dfAll['GDP_Partner'])
dfAll

Country  GDP_Reporter Partner   GDP_Partner        Trade     Distance  \
ALB 1      ALB  1.514702e+10     AUT  4.550949e+11     79968.33   841.882811   
    3      ALB  1.514702e+10     BEL  5.437344e+11     37387.30  1600.602999   
    5      ALB  1.514702e+10     BGR  6.623016e+10    109838.63   461.650831   
    4      ALB  1.514702e+10     BIH  2.018351e+10     36665.88   374.025752   
    2      ALB  1.514702e+10     BLR  6.003126e+10       194.90  1493.866592   
...        ...           ...     ...           ...          ...          ...   
UKR 30     UKR  1.309019e+11     RUS  1.669583e+12  11742819.78  1638.390411   
    32     UKR  1.309019e+11     SRB  5.064065e+10    430954.93   967.319739   
    33     UKR  1.309019e+11     SVK  1.057019e+11   1389805.76   921.085735   
    34     UKR  1.309019e+11     SVN  5.416164e+10    223816.12  1296.938407   
    36     UKR  1.309019e+11     SWE  5.554554e+11    535669.39  1537.059179   

        const    l_Trade  l_Distance  l_GDP_Reporter  l_GDP_Partner  
ALB 1       1  11.289386    6.735641       23.441070      26.843772  
    3       1  10.529086    7.378136       23.441070      27.021727  
    5       1  11.606768    6.134809       23.441070      24.916402  
    4       1  10.509602    5.924325       23.441070      23.728132  
    2       1   5.272487    7.309123       23.441070      24.818131  
...       ...        ...         ...             ...            ...  
UKR 30      1  16.278753    7.401470       25.597714      28.143595  
    32      1  12.973759    6.874529       25.597714      24.648020  
    33      1  14.144675    6.825553       25.597714      25.383889  
    34      1  12.318580    7.167762       25.597714      24.715239  
    36      1  13.191272    7.337626       25.597714      27.043054  

[1560 rows x 11 columns]

In [19]:
dfAll #run to view the dataframe in its final state

Country  GDP_Reporter Partner   GDP_Partner        Trade     Distance  \
ALB 1      ALB  1.514702e+10     AUT  4.550949e+11     79968.33   841.882811   
    3      ALB  1.514702e+10     BEL  5.437344e+11     37387.30  1600.602999   
    5      ALB  1.514702e+10     BGR  6.623016e+10    109838.63   461.650831   
    4      ALB  1.514702e+10     BIH  2.018351e+10     36665.88   374.025752   
    2      ALB  1.514702e+10     BLR  6.003126e+10       194.90  1493.866592   
...        ...           ...     ...           ...          ...          ...   
UKR 30     UKR  1.309019e+11     RUS  1.669583e+12  11742819.78  1638.390411   
    32     UKR  1.309019e+11     SRB  5.064065e+10    430954.93   967.319739   
    33     UKR  1.309019e+11     SVK  1.057019e+11   1389805.76   921.085735   
    34     UKR  1.309019e+11     SVN  5.416164e+10    223816.12  1296.938407   
    36     UKR  1.309019e+11     SWE  5.554554e+11    535669.39  1537.059179   

        const    l_Trade  l_Distance  l_GDP_Reporter  l_GDP_Partner  
ALB 1       1  11.289386    6.735641       23.441070      26.843772  
    3       1  10.529086    7.378136       23.441070      27.021727  
    5       1  11.606768    6.134809       23.441070      24.916402  
    4       1  10.509602    5.924325       23.441070      23.728132  
    2       1   5.272487    7.309123       23.441070      24.818131  
...       ...        ...         ...             ...            ...  
UKR 30      1  16.278753    7.401470       25.597714      28.143595  
    32      1  12.973759    6.874529       25.597714      24.648020  
    33      1  14.144675    6.825553       25.597714      25.383889  
    34      1  12.318580    7.167762       25.597714      24.715239  
    36      1  13.191272    7.337626       25.597714      27.043054  

[1560 rows x 11 columns]

### Running our Regression

- Using the statsmodel module, we create a Ordinary Least Squares model, using 'l_Trade' as our endogenous variable, 'l_Distance', 'l_GDP_Reporter', and 'l_GDP_Partner' as our exogenous variables.

In [20]:
reg1 = sm.OLS(endog=dfAll['l_Trade'], exog=dfAll[['const', 'l_Distance', 'l_GDP_Reporter', 'l_GDP_Partner']], \
missing='drop')

results = reg1.fit()

type(results)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                l_Trade   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.862
Method:                 Least Squares   F-statistic:                     3252.
Date:                Fri, 05 Feb 2021   Prob (F-statistic):               0.00
Time:                        19:51:14   Log-Likelihood:                -2163.8
No. Observations:                1556   AIC:                             4336.
Df Residuals:                    1552   BIC:                             4357.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -25.0108      0.584    -42.

- We find all variables from our model to be highly significant. 
<br>As expected, the GDP of both the Reporter and Partner countries have a positive effect on trade, while distance has a negative effect.
- We save the coefficients from our model

In [21]:
const = -25.0108 # Constant
l_Distance = -1.6009 #If Distance goes up by 1%, then Trade goes down by -0.3158%
l_GDP_Reporter = 0.9598  #If GDP_Reporter goes up by 1%, then Trade goes up by 0.9545%
l_GDP_Partner = 0.9624 #If Distance goes up by 1%, then Trade goes up by 0.9616%

With our estimated predictors, we can now compare our trade with our fitted values. We do so by Creating a new column for Predicted_Trade:
$$
    Predicted\_Trade\ =\ \beta _{0} +\ \beta _{1} \ l\_distance \ \ +\ \beta _{2} \ l\_GDP\_Reporter\ +\ \beta _{3} \ l\_GDP\_Partner +\ \epsilon 
$$
And create a new column for Residual Trade:
$$
    Residual\_Trade\ =\ l\_Trade\ -Predicted\_Trade
$$
By sorting our dataframe with this value, we can now rank each country's level of trade with each other. In our example below we will be using the point of view of the Netherlands.

In [22]:
dfAll
reporter = 'NLD'
dfResults = dfAll[['Country','Partner','l_Trade','l_Distance','l_GDP_Reporter','l_GDP_Partner']].loc[dfAll['Country'] == reporter]
dfResults
predictedTrade = const + l_Distance * dfAll['l_Distance'] + l_GDP_Reporter * dfAll['l_GDP_Reporter'] + l_GDP_Partner * dfAll['l_GDP_Partner']
dfResults['Predicted_Trade'] = predictedTrade
dfResults['Residual_Trade'] = dfResults['l_Trade'] - dfResults['Predicted_Trade'] 
dfResults.sort_values('Residual_Trade', ascending=False, inplace=True)
dfResults

Country Partner    l_Trade  l_Distance  l_GDP_Reporter  l_GDP_Partner  \
NLD 16     NLD     ISL  14.580724    7.615629       27.541144      23.971219   
    7      NLD     CYP  13.368120    7.978975       27.541144      23.954458   
    23     NLD     MLT  13.304309    7.569951       27.541144      23.414563   
    30     NLD     RUS  16.959447    8.074737       27.541144      28.143595   
    11     NLD     FIN  16.157251    7.377384       27.541144      26.343473   
    10     NLD     EST  14.152803    7.300007       27.541144      24.145283   
    5      NLD     BGR  14.502828    7.498870       27.541144      24.916402   
    20     NLD     LTU  14.689054    7.147889       27.541144      24.707105   
    27     NLD     PRT  15.616293    7.473804       27.541144      26.213497   
    15     NLD     HUN  15.830177    7.041829       27.541144      25.801054   
    19     NLD     LVA  14.057925    7.204487       27.541144      24.261788   
    38     NLD     UKR  14.722176    7.548162       27.541144      25.597714   
    14     NLD     GRC  14.990986    7.621538       27.541144      26.108395   
    29     NLD     ROM  15.403441    7.389553       27.541144      26.209959   
    26     NLD     POL  17.045882    6.868990       27.541144      27.098992   
    35     NLD     ESP  17.106216    7.331487       27.541144      27.983194   
    8      NLD     CZE  16.511579    6.639796       27.541144      26.240352   
    25     NLD     NOR  16.523729    6.948953       27.541144      26.796694   
    22     NLD     MKD  12.313768    7.426383       27.541144      23.263534   
    36     NLD     SWE  16.759020    6.901873       27.541144      27.043054   
    33     NLD     SVK  15.004473    6.959291       27.541144      25.383889   
    32     NLD     SRB  13.743168    7.227781       27.541144      24.648020   
    28     NLD     MDA  11.900554    7.471622       27.541144      23.161842   
    34     NLD     SVN  14.354275    6.864277       27.541144      24.715239   
    17     NLD     IRL  16.358167    6.761940       27.541144      26.670450   
    4      NLD     BIH  12.701027    7.150037       27.541144      23.728132   
    31     NLD     MNE  11.180370    7.312751       27.541144      22.428787   
    18     NLD     ITA  17.324053    7.041645       27.541144      28.368924   
    6      NLD     HRV  13.838071    7.002586       27.541144      24.840272   
    1      NLD     AUT  16.175904    6.711855       27.541144      26.843772   
    2      NLD     BLR  13.129302    7.324520       27.541144      24.818131   
    0      NLD     ALB  11.624425    7.402760       27.541144      23.441070   
    9      NLD     DNK  16.289849    6.322162       27.541144      26.600664   
    12     NLD     FRA  17.956233    6.365200       27.541144      28.656297   
    13     NLD     DEU  19.156249    5.810754       27.541144      29.008216   
    3      NLD     BEL  18.498519    5.000065       27.541144      27.021727   
    39     NLD     GBR  18.115103    6.170042       27.541144      28.682076   
    37     NLD     CHE  16.082734    6.385833       27.541144      27.281663   
    21     NLD     LUX  14.762138    5.618733       27.541144      24.984818   

        Predicted_Trade  Residual_Trade  
NLD 16        12.301230        2.279494  
    7         11.703420        1.664700  
    23        11.838630        1.465679  
    30        15.581739        1.377708  
    11        14.965695        1.191556  
    10        12.974030        1.178774  
    5         13.397794        1.105035  
    20        13.758253        0.930801  
    27        14.686246        0.930047  
    15        14.980860        0.849318  
    19        13.239071        0.818854  
    38        13.974577        0.747600  
    14        14.348589        0.642397  
    29        14.817718        0.585723  
    26        16.506694        0.539188  
    35        16.617238        0.488977  
    8         16.047255        0.464324  
    25        16.087750        0.435979  
    22  

In order to generate a list of the best trade partners for each country, we now replicate the code above, looping all countries through a for loop.
Additionally, as we are only interested in each country's best trade partner, we will only append the top result from each country.

In [23]:
dfbest = []
for reporter in countries:
    dfResults = dfAll[['Country','Partner','l_Trade','l_Distance','l_GDP_Reporter','l_GDP_Partner']].loc[dfAll['Country'] == reporter]
    predictedTrade = const + l_Distance * dfAll['l_Distance'] + l_GDP_Reporter * dfAll['l_GDP_Reporter'] + l_GDP_Partner * dfAll['l_GDP_Partner']
    dfResults['Predicted_Trade'] = predictedTrade
    dfResults['Residual_Trade'] = dfResults['l_Trade'] - dfResults['Predicted_Trade'] 
    dfResults.sort_values('Residual_Trade', ascending=False, inplace=True)
    dfbest.append(dfResults[0:1])
dfbest = pd.concat(dfbest)
# dfbest.sort_values('Partner', inplace=True) used this line to construct the map more efficiently
dfbest

,,Country,Partner,l_Trade,l_Distance,l_GDP_Reporter,l_GDP_Partner,Predicted_Trade,Residual_Trade
ALB,32,ALB,SRB,13.319996,5.996315,23.441070,24.648020,11.609693,1.710303
AUT,16,AUT,ISL,11.820356,7.942509,26.843772,23.971219,11.108591,0.711765
BEL,5,BEL,BGR,14.572154,7.497143,27.021727,24.916402,12.902022,1.670132
BGR,23,BGR,MLT,12.008557,7.080805,24.916402,23.414563,10.102476,1.906080
BIH,31,BIH,MNE,12.522946,5.427877,23.728132,22.428787,10.659438,1.863508
BLR,30,BLR,RUS,17.379549,7.450541,24.818131,28.143595,13.967468,3.412081
CHE,17,CHE,IRL,16.139072,7.158639,27.281663,26.670450,15.381517,0.757555
CYP,14,CYP,GRC,14.628538,6.880153,23.954458,26.108395,12.092972,2.535565
CZE,22,CZE,MKD,13.101396,6.916126,26.240352,23.263534,11.491489,1.609907
DEU,22,DEU,MKD,15.396908,7.214334,29.008216,23.263534,13.670683,1.726225


We can replicate the steps above, this time, showing the **worst** trade partner for each country:

In [24]:
dfworst = []
for reporter in countries:
    dfResults = dfAll[['Country','Partner','l_Trade','l_Distance','l_GDP_Reporter','l_GDP_Partner']].loc[dfAll['Country'] == reporter]
    predictedTrade = const + l_Distance * dfAll['l_Distance'] + l_GDP_Reporter * dfAll['l_GDP_Reporter'] + l_GDP_Partner * dfAll['l_GDP_Partner']
    dfResults['Predicted_Trade'] = predictedTrade
    dfResults['Residual_Trade'] = dfResults['l_Trade'] - dfResults['Predicted_Trade'] 
    dfResults.sort_values('Residual_Trade', ascending=False, inplace=True)
    dfworst.append(dfResults[38:39])
dfworst = pd.concat(dfworst)
# dfworst.sort_values('Partner', inplace=True)
dfworst

,,Country,Partner,l_Trade,l_Distance,l_GDP_Reporter,l_GDP_Partner,Predicted_Trade,Residual_Trade
ALB,2,ALB,BLR,5.272487,7.309123,23.441070,24.818131,9.671733,-4.399247
AUT,2,AUT,BLR,12.040271,7.026930,26.843772,24.818131,13.389409,-1.349138
BEL,0,BEL,ALB,10.677988,7.378136,27.021727,23.441070,11.672681,-0.994693
BGR,25,BGR,NOR,11.389347,7.730598,24.916402,26.796694,12.317187,-0.927841
BIH,16,BIH,ISL,5.946938,8.096019,23.728132,23.971219,7.872445,-1.925507
BLR,0,BLR,ALB,6.625525,7.309123,24.818131,23.441070,9.668153,-3.042628
CHE,21,CHE,LUX,13.585251,5.776687,27.281663,24.984818,15.971631,-2.386380
CYP,2,CYP,BLR,7.121301,7.653889,23.954458,24.818131,9.612547,-2.491246
CZE,37,CZE,CHE,15.315813,6.469341,26.240352,27.281663,16.073795,-0.757982
DEU,21,DEU,LUX,16.222422,5.666418,29.008216,24.984818,17.805306,-1.582883


# Sensitivity analysis

Before finally deciding to run a simple log-log OLS model I have considered several alternatives.

Firstly, I considered running a WLS regressions, using each reporting country's GDP (current USD) as the weights.

In [25]:
reg1 = sm.WLS(endog=dfAll['l_Trade'], exog=dfAll[['const', 'l_Distance', 'l_GDP_Reporter', 'l_GDP_Partner']],weights=dfAll['l_GDP_Reporter'], \
missing='drop')

results = reg1.fit()

type(results)
print(results.summary())

                            WLS Regression Results                            
Dep. Variable:                l_Trade   R-squared:                       0.864
Model:                            WLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     3284.
Date:                Fri, 05 Feb 2021   Prob (F-statistic):               0.00
Time:                        19:51:15   Log-Likelihood:                -2153.9
No. Observations:                1556   AIC:                             4316.
Df Residuals:                    1552   BIC:                             4337.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -25.0084      0.583    -42.

However, I was not convinced that adding more weights to bigger countries was necessarily a good idea. 
But more importantly, by comparing our parameters with our previous OLS model, we can confirm that the weights did not affect the model's coefficients in a significant way.
Therefore, I decided to keep the model simple by using an OLS regression.

Another idea I experimented with was the functional form of our model.
After adding our exogenous variable in their nominal form (distance, GDP_Reporter, and GDP_Partner) as well as other forms such as ln(distance^2) I saw that all of these were statistically significant.
And while this time the coefficients changed, after calculating the residual_trade for each country, I observed that the country ranks did not change significantly.

Therefore, as the result did not change, I decided to stick with the traditional approach:

$$
    l\_Trade\ =\ \ \beta _{0} +\ \beta _{1} \ l\_distance \ \ +\ \beta _{2} \ l\_GDP\_Reporter\ +\ \beta _{3} \ l\_GDP\_Partner\ +\ \epsilon 
$$

# Discussion and conclusion

There are many interesting observation points we can point out through this ranking system.

In the case of the Netherlands, we see that its top three trade partners are small island nations. A possible explanation for this is the importance of the Rotterdam port. 
As Rotterdam has the biggest port in Europe and the small island nations of Iceland, Cyprus, and Malta largely depend on trade through naval routes, it is likely that these countries are trading with mainland Europe through the port of Rotterdam, and therefore, their trade with the Netherlands is inflated. 

In the case of Russia, Netherland's fourth trade partner, its relationship is based on the oil industry, which results in almost 90% of the Dutch imports from Russia. The Netherlands does not only import oil from Russia for their own consumption, but it also then sent to the rest of the continent through European supply pipes or sent to the rest of the world through bulk carriers. 
source:https://www.researchgate.net/publication/323417461_Economic_relations_between_the_Netherlands_and_Russia

We can display the dataframe of the best trade partners per country with the following map:

## Best Trade Partner per Country (after controlling for economic size and distance):  
<img src="trademap.png">

The map above shows a great variety of countries; however, we can identify a few themes.

Central and Nordic Europe are dominated by Iceland and Macedonia. In the case of Iceland, it is simple to think that Iceland's best trade partners are those who have ports in the North Sea. However, it is challenging to find a connection between Macedonia and Czechia, Germany, Slovenia, and the United Kingdom. 

The prevalence of small countries such as Cyprus, Iceland, and Malta might be result of a bias within the model. A possible idea to rectify this issue would be to include other geographical data within the model. Controlling for geographical features which impact trade such as being an island or a landlocked country as well as the roughness of the terrain in a country's borders (Alps, Pyrenees).

We can observe some economic isolation within The Ex-Yugoslav countries, every single country has another Ex-Yugoslav country as their best trading partner. The only other place where we observe a similar cluster is in the Baltic region, where both Latvia and Lithuania have Estonia as their best trading partner while Estonia itself has Lithuania as its top trading partner. In both cases, this might be explained by their shared recent history, as most of these countries did not exist separate from each other until the 1990's. 

Nevertheless, this map shows much greater variety over the first map shown in the introduction. After controlling for economic size and distance, we observe a much different picture, however, while we can still spot several patterns, it would be extremely challenging to thoroughly explain every single outcome.

Finally, this ranking system can be useful in order to determine both the best trading partners of a country.
However, it could be even more beneficial to identify possible opportunities for a country by trying to understand why they have a poor rank for a specific trading partner. Through a policy point of view, a country might want to allocate resources into growing their trading relationship with another country.